In [ ]:
import sqlite3
conn=sqlite3.connect("onthi.db")
cursor=conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXITS CUAHANG(
    CUAHANG(MACH,TenCH,GioMoCua,GioDongCua)
)
               
               """)

In [2]:
import sqlite3
from datetime import date

# 1) Kết nối SQLite
conn = sqlite3.connect("restaurant.db")
conn.execute("PRAGMA foreign_keys = ON;")
cur = conn.cursor()

# 2) Xóa bảng cũ (nếu chạy nhiều lần)
cur.executescript("""
DROP TABLE IF EXISTS DANHGIA;
DROP TABLE IF EXISTS DATMON;
DROP TABLE IF EXISTS KHACHHANG;
DROP TABLE IF EXISTS MONAN;
DROP TABLE IF EXISTS LOAIMON;
DROP TABLE IF EXISTS CUAHANG;
""")

# 3) Tạo bảng kèm ràng buộc
cur.executescript("""
CREATE TABLE CUAHANG(
  MaCH INTEGER PRIMARY KEY,
  TenCH TEXT NOT NULL UNIQUE,
  GioMoCua TEXT NOT NULL,
  GioDongCua TEXT NOT NULL
);

CREATE TABLE LOAIMON(
  MaLM INTEGER PRIMARY KEY,
  TenLM TEXT NOT NULL UNIQUE
);

CREATE TABLE MONAN(
  MaMA INTEGER PRIMARY KEY,
  TenMA TEXT NOT NULL,
  MaLM INTEGER NOT NULL,
  GiaBan REAL NOT NULL CHECK(GiaBan > 0),
  FOREIGN KEY (MaLM) REFERENCES LOAIMON(MaLM) ON DELETE RESTRICT
);

CREATE TABLE KHACHHANG(
  MaKH INTEGER PRIMARY KEY,
  HoTen TEXT NOT NULL
);

CREATE TABLE DATMON(
  MaDM INTEGER PRIMARY KEY,
  MaMA INTEGER NOT NULL,
  SoLuong INTEGER NOT NULL CHECK(SoLuong > 0),
  FOREIGN KEY (MaMA) REFERENCES MONAN(MaMA) ON DELETE CASCADE
);

CREATE TABLE DANHGIA(
  MaDG INTEGER PRIMARY KEY,
  MaKH INTEGER NOT NULL,
  MaCH INTEGER NOT NULL,
  SoSao INTEGER NOT NULL CHECK(SoSao BETWEEN 0 AND 5),
  NgayDanhGia DATE NOT NULL,
  FOREIGN KEY (MaKH) REFERENCES KHACHHANG(MaKH) ON DELETE CASCADE,
  FOREIGN KEY (MaCH) REFERENCES CUAHANG(MaCH) ON DELETE CASCADE
);
""")

# 4) Dữ liệu mẫu (5 cửa hàng, 10 loại, 15 món, 15 đơn, 12 đánh giá)
cuahang = [
    (1, "PHO 24",        "07:00", "22:00"),
    (2, "Cơm Tấm Cali",  "08:00", "21:30"),
    (3, "Bún Chả Hà Nội","09:00", "20:00"),
    (4, "Pizza 4P's",    "10:00", "23:00"),
    (5, "Highlands",     "06:00", "22:30")
]

loaimon = [(i, f"Loại {i}") for i in range(1, 11)]

monan = [
    (1,  "Phở Bò",            1, 45000),
    (2,  "Phở Gà",            1, 42000),
    (3,  "Cơm Tấm Sườn",      2, 55000),
    (4,  "Cơm Tấm Bì",        2, 50000),
    (5,  "Bún Chả",           3, 60000),
    (6,  "Nem Rán",           3, 30000),
    (7,  "Pizza Margherita",  4,150000),
    (8,  "Pizza Pepperoni",   4,170000),
    (9,  "Cà Phê Sữa",        5, 35000),
    (10, "Cà Phê Đen",        5, 30000),
    (11, "Trà Đào",           6, 40000),
    (12, "Trà Vải",           6, 40000),
    (13, "Bánh Mì Thịt",      7, 25000),
    (14, "Bánh Mì Ốp La",     7, 27000),
    (15, "Bánh Bao",          8, 20000)
]

khachhang = [(i, f"Khách {i}") for i in range(1, 13)]

# 15 đơn (MaDM 1-15) – mỗi đơn ứng với 1 món để đơn giản
datmon = [(i, (i % 15) + 1, (i % 3) + 1) for i in range(1, 16)]

# 12 đánh giá
danhgia = []
from datetime import date, timedelta
...
for i in range(1, 13):
    danhgia.append((
        i,
        i,
        (i % 5) + 1,
        i % 6,
        (date(2025, 6, 8) - timedelta(days=i)).isoformat()
    ))


# 5) Chèn dữ liệu
cur.executemany("INSERT INTO CUAHANG VALUES (?,?,?,?)", cuahang)
cur.executemany("INSERT INTO LOAIMON VALUES (?,?)", loaimon)
cur.executemany("INSERT INTO MONAN VALUES (?,?,?,?)",  monan)
cur.executemany("INSERT INTO KHACHHANG VALUES (?,?)",  khachhang)
cur.executemany("INSERT INTO DATMON VALUES (?,?,?)",   datmon)
cur.executemany("INSERT INTO DANHGIA VALUES (?,?,?,?,?)", danhgia)

conn.commit()
print("Đã khởi tạo xong cơ sở dữ liệu!")
conn.close()


Đã khởi tạo xong cơ sở dữ liệu!


In [4]:
import sqlite3

# Kết nối
conn = sqlite3.connect("restaurant.db")
cur = conn.cursor()

# Truy vấn doanh thu theo cửa hàng và loại món
query = """
SELECT 
    CH.TenCH,
    LM.TenLM,
    SUM(DM.SoLuong * MA.GiaBan) AS DoanhThu
FROM DATMON DM
JOIN MONAN MA ON DM.MaMA = MA.MaMA
JOIN LOAIMON LM ON MA.MaLM = LM.MaLM
-- Gán tạm cửa hàng theo modulo (MaMA % 5 + 1)
JOIN CUAHANG CH ON CH.MaCH = ((MA.MaMA - 1) % 5 + 1)
GROUP BY CH.TenCH, LM.TenLM
ORDER BY CH.TenCH, DoanhThu DESC;
"""

cur.execute(query)
rows = cur.fetchall()

# In kết quả
print("{:<20} {:<15} {:>10}".format("CuaHang", "LoaiMon", "DoanhThu"))
print("-" * 50)
for row in rows:
    print("{:<20} {:<15} {:>10,.0f}".format(row[0], row[1], row[2]))

conn.close()


CuaHang              LoaiMon           DoanhThu
--------------------------------------------------
Bún Chả Hà Nội       Loại 4             340,000
Bún Chả Hà Nội       Loại 2             165,000
Bún Chả Hà Nội       Loại 7              25,000
Cơm Tấm Cali         Loại 4             150,000
Cơm Tấm Cali         Loại 6             120,000
Cơm Tấm Cali         Loại 1              84,000
Highlands            Loại 3             120,000
Highlands            Loại 8              60,000
Highlands            Loại 5              30,000
PHO 24               Loại 3              90,000
PHO 24               Loại 6              80,000
PHO 24               Loại 1              45,000
Pizza 4P's           Loại 5             105,000
Pizza 4P's           Loại 7              54,000
Pizza 4P's           Loại 2              50,000


In [5]:
import sqlite3

# Kết nối CSDL
conn = sqlite3.connect("restaurant.db")
cur = conn.cursor()

# Truy vấn trung bình số sao và phân loại
query = """
SELECT 
    CH.TenCH,
    ROUND(AVG(DG.SoSao), 2) AS TrungBinhSao,
    CASE 
        WHEN AVG(DG.SoSao) < 3.5 THEN 'Cần cải thiện'
        WHEN AVG(DG.SoSao) <= 4.5 THEN 'Tốt'
        ELSE 'Xuất sắc'
    END AS PhanLoai
FROM DANHGIA DG
JOIN CUAHANG CH ON DG.MaCH = CH.MaCH
GROUP BY DG.MaCH
ORDER BY TrungBinhSao DESC;
"""

# Thực hiện truy vấn
cur.execute(query)
results = cur.fetchall()

# In kết quả
print("{:<25} {:>15} {:<20}".format("Tên Cửa Hàng", "TB Số Sao", "Phân Loại"))
print("-" * 60)
for ten, tb_sao, loai in results:
    print("{:<25} {:>15} {:<20}".format(ten, tb_sao, loai))

conn.close()


Tên Cửa Hàng                    TB Số Sao Phân Loại           
------------------------------------------------------------
PHO 24                                4.5 Tốt                 
Highlands                             3.5 Tốt                 
Pizza 4P's                            2.5 Cần cải thiện       
Cơm Tấm Cali                          2.0 Cần cải thiện       
Bún Chả Hà Nội                        1.0 Cần cải thiện       


In [6]:
import sqlite3
import numpy as np

# Kết nối
conn = sqlite3.connect("restaurant.db")
cur = conn.cursor()

# Lấy toàn bộ SoLuong
cur.execute("SELECT SoLuong FROM DATMON")
soluongs = [row[0] for row in cur.fetchall()]

# Tính Q1, Q3, IQR
q1 = np.percentile(soluongs, 25)
q3 = np.percentile(soluongs, 75)
iqr = q3 - q1

# Ngưỡng phát hiện ngoại lai
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

print(f"Q1 = {q1}, Q3 = {q3}, IQR = {iqr}")
print(f"Ngưỡng dưới = {lower_bound}, Ngưỡng trên = {upper_bound}")

# Truy vấn các bản ghi ngoại lai
cur.execute("""
SELECT * FROM DATMON 
WHERE SoLuong < ? OR SoLuong > ?
""", (lower_bound, upper_bound))

outliers = cur.fetchall()
print("\n🔍 Các bản ghi ngoại lai:")
for row in outliers:
    print(row)


Q1 = 1.0, Q3 = 3.0, IQR = 2.0
Ngưỡng dưới = -2.0, Ngưỡng trên = 6.0

🔍 Các bản ghi ngoại lai:
